In [20]:
print('jmd shree ganesha')

jmd shree ganesha


In [21]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain_community.vectorstores import chroma
from langchain_community.llms import llamacpp
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
import os

In [22]:
import os
print(os.getcwd())

/home/aster/FAQ-Chatbot-ChromaDB/notebook


In [23]:
dataset = r"/home/aster/FAQ-Chatbot-ChromaDB/data"

In [24]:
memory = ConversationBufferMemory(input_key="question", memory_key="history")

In [25]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [26]:
extracted_data = load_pdf(dataset)

In [27]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [28]:
text_split_chunks = text_splitter(extracted_data)

In [29]:
text_split_chunks

[Document(page_content='Yes, you can book appointments and many treatments directly with us. For most procedures, a \nGP referral is required.', metadata={'source': '/home/aster/FAQ-Chatbot-ChromaDB/data/dataset.pdf', 'page': 0}),
 Document(page_content='If you are insured, we also recommend that you check with your insurer that you are covered to \nsee your preferred consultant at our hospital.', metadata={'source': '/home/aster/FAQ-Chatbot-ChromaDB/data/dataset.pdf', 'page': 0}),
 Document(page_content='You can easily book an appointment online with many of our consultants. To find a consultant', metadata={'source': '/home/aster/FAQ-Chatbot-ChromaDB/data/dataset.pdf', 'page': 0}),
 Document(page_content='near you, please clic k here. For more information, please call 0330 173 8200.', metadata={'source': '/home/aster/FAQ-Chatbot-ChromaDB/data/dataset.pdf', 'page': 0}),
 Document(page_content='We are proud to work with expert health professionals who specialize in a wide range of', met

In [30]:
print(len(text_split_chunks))

84


In [31]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [32]:
embeddings = text_to_embeddings()

/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [34]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [35]:
query_result

[-0.03447726368904114,
 0.031023193150758743,
 0.0067349947057664394,
 0.02610894665122032,
 -0.03936201333999634,
 -0.16030244529247284,
 0.06692396849393845,
 -0.006441481877118349,
 -0.04745052754878998,
 0.014758824370801449,
 0.07087526470422745,
 0.05552756413817406,
 0.019193341955542564,
 -0.026251332834362984,
 -0.010109550319612026,
 -0.026940450072288513,
 0.022307418286800385,
 -0.022226607427001,
 -0.1496926248073578,
 -0.01749303564429283,
 0.007676276378333569,
 0.0543522946536541,
 0.00325443921610713,
 0.03172590211033821,
 -0.08462145179510117,
 -0.0294059868901968,
 0.051595594733953476,
 0.048124048858881,
 -0.003314807778224349,
 -0.05827918276190758,
 0.04196925461292267,
 0.022210685536265373,
 0.1281888484954834,
 -0.02233896590769291,
 -0.01165627222508192,
 0.06292841583490372,
 -0.03287633880972862,
 -0.09122608602046967,
 -0.031175406649708748,
 0.052699532359838486,
 0.04703480750322342,
 -0.08420310169458389,
 -0.030056176707148552,
 -0.02074483036994934,


In [36]:
persist_directory = 'chat_db'

vectordb = chroma.Chroma.from_documents(documents=text_split_chunks, embedding=embeddings, persist_directory=persist_directory)

In [37]:
vectordb.persist()
vectordb = None

In [38]:
vectordb = chroma.Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [39]:
retriever = vectordb.as_retriever()

In [40]:
docsearch = retriever.get_relevant_documents('Do I need a referral from my GP?')

/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [41]:
len(docsearch)
print(docsearch)

[Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': '/home/aster/FAQ-Chatbot-ChromaDB/data/dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'}), Document(page_content='A GP referral is required for most procedures as this will ensure that you are booked in for the', metadata={'page': 0, 'source': 'C:\\Users\\Admin\\Desktop\\Aster\\FAQ Chatbot\\FAQ-Chatbot\\data\\dataset.pdf'})]


In [42]:
retriever = vectordb.as_retriever(search_kwargs={'k':4})

In [43]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a helpful assistant who answers questions related to Nuffield Health Hospital.
If the question is not related to Nuffield Health, respond with "I don't know, It's out of my knowledge base.", 
else if you don't know the answer, just say that to contact the helpline number 0330 173 8200. Not to ask for the phone number.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [44]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [45]:
chain_type_kwargs={"prompt": PROMPT}

In [46]:
llm2 = llamacpp.LlamaCpp(model_path=r"/home/aster/FAQ-Chatbot-ChromaDB/model/llama-2-7b-chat.Q5_K_M.gguf.bin", model_type="llama", n_ctx=4096, n_gpu_layers=-1, n_threads=4, retriever=retriever, verbose=True, temperature=0.5, context_length=4096, max_tokens=2048)

/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model_type is not default parameter.
                model_type was transferred to model_kwargs.
                Please confirm that model_type is what you intended.
  warnings.warn(
/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! retriever is not default parameter.
                retriever was transferred to model_kwargs.
                Please confirm that retriever is what you intended.
  warnings.warn(
/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! context_length is not default parameter.
                context_length was transferred to model_kwargs.
                Please confirm that context_length is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and

In [47]:
qa=RetrievalQA.from_chain_type(
    llm=llm2, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=False, 
    chain_type_kwargs=chain_type_kwargs)

In [48]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/home/aster/FAQ-Chatbot-ChromaDB/chroma/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    5177.24 ms
llama_print_timings:      sample time =      13.23 ms /    63 runs   (    0.21 ms per token,  4762.98 tokens per second)
llama_print_timings: prompt eval time =   39392.98 ms /   189 tokens (  208.43 ms per token,     4.80 tokens per second)
llama_print_timings:        eval time =   19778.16 ms /    62 runs   (  319.00 ms per token,     3.13 tokens per second)
llama_print_timings:       total time =   59449.81 ms /   251 tokens


Response :  You do indeed require a GP referral for most procedures at Nuffield Health Hospital. Please speak to your GP to obtain a referral before booking an appointment. Alternatively, you can contact our helpline on 0330 173 8200 for more information.


Llama.generate: prefix-match hit

llama_print_timings:        load time =    5177.24 ms
llama_print_timings:      sample time =      12.62 ms /    56 runs   (    0.23 ms per token,  4439.16 tokens per second)
llama_print_timings: prompt eval time =   14655.48 ms /    74 tokens (  198.05 ms per token,     5.05 tokens per second)
llama_print_timings:        eval time =   16764.45 ms /    55 runs   (  304.81 ms per token,     3.28 tokens per second)
llama_print_timings:       total time =   31618.29 ms /   129 tokens


Response :  Bon Jovi is not a doctor or medical professional at Nuffield Health Hospital. If you have any questions about our hospital or treatment options, please don't hesitate to contact us on 0330 173 8200.


Llama.generate: prefix-match hit

llama_print_timings:        load time =    5177.24 ms
llama_print_timings:      sample time =       8.36 ms /    41 runs   (    0.20 ms per token,  4903.13 tokens per second)
llama_print_timings: prompt eval time =   15821.55 ms /    62 tokens (  255.19 ms per token,     3.92 tokens per second)
llama_print_timings:        eval time =   12208.48 ms /    40 runs   (  305.21 ms per token,     3.28 tokens per second)
llama_print_timings:       total time =   28163.59 ms /   102 tokens


Response :  The consultation fee for a specific consultant can be found on their profile page or by contacting our helpline number 0330 173 8200.


Llama.generate: prefix-match hit

llama_print_timings:        load time =    5177.24 ms
llama_print_timings:      sample time =      22.19 ms /   107 runs   (    0.21 ms per token,  4822.43 tokens per second)
llama_print_timings: prompt eval time =   13699.91 ms /    67 tokens (  204.48 ms per token,     4.89 tokens per second)
llama_print_timings:        eval time =   32396.55 ms /   106 runs   (  305.63 ms per token,     3.27 tokens per second)
llama_print_timings:       total time =   46428.81 ms /   173 tokens


Response :  Yes, we do have a visitors' policy in place at our hospital. Visitors are allowed between 2pm and 8pm every day, except for certain areas of the hospital where visiting is restricted for privacy or safety reasons. Please check with the nursing staff on duty for more information.
I don't know, It's out of my knowledge base.
To contact the helpline number, please click here 0330 173 8200.


Llama.generate: prefix-match hit
